In [1]:
import cv2
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import pytesseract
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service 
import re
import time

In [2]:
ds_xa = [8051905]

index = 0

def get_captcha(driver):
    # Lấy bản chụp màn hình dưới dạng dữ liệu PNG
    screenshot = driver.get_screenshot_as_png()
    # driver.save_screebshot("screenshot.png")
    # cv2.imwrite("screenshot.png", screenshot)
    # Chuyển đổi dữ liệu PNG thành mảng NumPy
    screenshot_array = np.frombuffer(screenshot, np.uint8)

    # Đọc hình ảnh từ mảng NumPy
    image = cv2.imdecode(screenshot_array, cv2.IMREAD_COLOR)

    # Cắt ảnh
    x1, y1, x2, y2 = (815, 668, 990, 735)
    crop = image[y1:y2, x1:x2]
    cv2.imwrite("crop.png", crop)

    resized_image = cv2.resize(crop, (390, 150))
    blurred_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)

    kernel = np.ones((6,6), np.uint8)
    dilation = cv2.dilate(blurred_image, kernel, iterations=1)

    for row in range(len(dilation)):
        for col in range(len(dilation[row])):
            if dilation[row, col] < 150:
                dilation[row, col] = 0
            else:
                dilation[row, col] = 255

    dilation = cv2.medianBlur(dilation, 5, 0)

    height, width = dilation.shape

    # Kích thước mới (tăng thêm 30 pixel ở dưới)
    new_height = height + 30
    new_width = width

    # Tạo một hình ảnh mới với kích thước mới và màu trắng
    new_image = np.zeros((new_height, new_width), dtype=np.uint8)
    new_image[:] = (255)  # Gán màu trắng (255, 255, 255)

    # Sao chép hình ảnh gốc vào hình ảnh mới
    new_image[:height, :] = dilation

    # Trích xuất văn bản từ hình ảnh
    text = pytesseract.image_to_string(new_image,nice = 20,config='--psm 13')

    #xóa các kí tự punctuation
    text = re.sub(r'[^\w\s]', '', text)

    # Loại bỏ các khoảng trắng thừa
    text = ''.join(text.split())

    return text

def test_captcha(driver,captcha):
    if len(captcha) != 5:
        return False
        
    url = f"https://www.gdt.gov.vn/TTHKApp/jsp/results.jsp?maTinh=805&maHuyen=80511&maXa=&hoTen=&kyLb=&diaChi=&maSoThue=&searchType=11&uuid=d01567b7-cb8b-47ae-b3b7-dfd6132076c4&captcha={captcha}&pageNumber=1"
    driver.get(url)
    element = driver.find_element(By.XPATH, '/html/body/div[1]')
    if element.text == "Vui lòng nhập đúng mã xác nhận.":
        return False
    driver.save_screenshot("screenshot.png")
    return True

def refresh_captcha(driver):
    time.sleep(1)
    driver.find_element(By.CSS_SELECTOR, '#nntSearchForm > div:nth-child(9) > div > img').click()
    
def main():
    global index
    pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
    path = r'C:\Users\Admin\PycharmProjects\Crawl_data_gov\chromedriver-win64\chromedriver.exe'
    service = Service(path)
    driver = webdriver.Chrome(service=service)

    # Truy cập trang web
    

    while True:
        driver.get("https://www.gdt.gov.vn/wps/portal/home/hct")
        time.sleep(1)
        captcha = get_captcha(driver)

        if test_captcha(driver,captcha):
            print(f"captcha is correct:{captcha}")
            # print(len(captcha))
            break
        # refresh_captcha(driver)
    

    # ds_xa = [1015149, 1015135, 1015139, 1015131, 1015121, 1015107, 1015123]
    for i in range(len(ds_xa)):
        xa = ds_xa[i]
        index = i
        xa = str(xa)
        tinh = str(xa[:3])
        huyen = str(xa[:5])
        search_types = {'11':'/html/body/div[4]/table',
                        '10':'/html/body/table',
                        '12':'/html/body/div[4]/table',
                        '03':'/html/body/div[4]/table',
                        '04':'/html/body/table'}
        
        search_types_name = {'11':'Danh sách hộ kinh doanh và mức thuế phải nộp',
                             '10':'Danh sách hộ kinh doanh không thuộc diện nộp thuế GTGT',
                             '12':'Danh sách hộ kinh doanh tạm nghỉ kinh doanh được miễn, giảm thuế',
                             '03':'Danh sách hộ kinh doanh điều chỉnh thông tin và tiền thuế',
                             '04':'Danh sách hộ kinh doanh nghỉ, bỏ kinh doanh'}
        
        for type in search_types:
            start_time = time.time()
            df = pd.DataFrame()
            for page in range(1,100):
                url = f"https://www.gdt.gov.vn/TTHKApp/jsp/results.jsp?maTinh={tinh}&maHuyen={huyen}&maXa={xa}&kyLb=&searchType={type}&captcha={captcha}&pageNumber={page}"
                driver.get(url)

                notify_element = driver.find_element(By.XPATH, '/html/body/div[1]')
                if notify_element.text == "Không tìm thấy kết quả":
                    break

                table_element = driver.find_element(By.XPATH,search_types[type])
                tables = pd.read_html(table_element.get_attribute("outerHTML"))
                if len(tables[0]) < 5:
                    break
                tables[0]['mã_tỉnh'] = tinh
                tables[0]['mã_huyện'] = huyen
                tables[0]['mã_xã'] = xa
                tables[0]['search_type'] = type
                df=pd.concat([df,tables[0]])
        # driver.save_screenshot(f"screenshot_{tinh}_{huyen}_{page}.png")
            path1 = f'C:\\Users\\Admin\\PycharmProjects\\Crawl_data_gov\\BIDV_Data\\{tinh}_{huyen}_{xa}_{type}.csv'
            df.to_csv(path1,index=False,encoding='utf-8-sig')
            print(f"Done {tinh}_{huyen}_{xa}_{type}.csv")
        
            end_time = time.time()
            elapsed_time = end_time - start_time
            log_file = open("log.txt", "a")
            timestamp = time.strftime("%Y-%m-%d %H:%M:%S")
            task_description = f"Task Crawl {xa}, Type: {search_types_name[type]}"
            log_message = f"{timestamp} - {task_description} took {elapsed_time:.2f} seconds to run\n"
            log_file.write(log_message)
            log_file.close()
    driver.quit()
    # return df


In [3]:
# Start the timer
start_time = time.time()

# Perform your task here
# For example, you can simulate a delay using time.sleep
time.sleep(2)

# Stop the timer
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

# Write to the log file
log_file = open("log.txt", "a")
timestamp = time.strftime("%Y-%m-%d %H:%M:%S")
task_description = "Task XYZ"
log_message = f"{timestamp} - {task_description} - took {elapsed_time:.2f} seconds to run\n"
log_file.write(log_message)
log_file.close()

In [4]:
while True:
    try:
        main()
        if index == len(ds_xa)-1:
            break           
    except:
        print(f"Error at {ds_xa[index]}")
        ds_xa = ds_xa[index:]

captcha is correct:m42nh
Done 805_80519_8051905_11.csv
Done 805_80519_8051905_10.csv
Done 805_80519_8051905_12.csv
Done 805_80519_8051905_03.csv
Done 805_80519_8051905_04.csv


In [5]:
search_types = {'11':'/html/body/div[4]/table',
                        '10':'/html/body/table',
                        '12':'/html/body/div[4]/table',
                        '03':'/html/body/div[4]/table',
                        '04':'/html/body/table'}

for type in search_types:
    print(type)

11
10
12
03
04


In [6]:
global index
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
path = r'C:\Users\Admin\PycharmProjects\Crawl_data_gov\chromedriver-win64\chromedriver.exe'
service = Service(path)
driver = webdriver.Chrome(service=service)

while True:
    driver.get("https://www.gdt.gov.vn/wps/portal/home/hct")
    time.sleep(1)
    captcha = get_captcha(driver)

    if test_captcha(driver,captcha):
        print(f"captcha is correct:{captcha}")
        # print(len(captcha))
        break

captcha is correct:6xf26


In [22]:
ds_xa = [8051905]
for i in range(len(ds_xa)):
        xa = ds_xa[i]
        index = i
        xa = str(xa)
        tinh = str(xa[:3])
        huyen = str(xa[:5])
        
        search_types = {'11':'/html/body/div[4]/table',
                        '10':'/html/body/table',
                        '12':'/html/body/div[4]/table',
                        '03':'/html/body/div[4]/table',
                        '04':'/html/body/table'}
        
        search_types_name = {'11':'Danh sách hộ kinh doanh và mức thuế phải nộp',
                             '10':'Danh sách hộ kinh doanh không thuộc diện nộp thuế GTGT',
                             '12':'Danh sách hộ kinh doanh tạm nghỉ kinh doanh được miễn, giảm thuế',
                             '03':'Danh sách hộ kinh doanh điều chỉnh thông tin và tiền thuế',
                             '04':'Danh sách hộ kinh doanh nghỉ, bỏ kinh doanh'}
        
        for type in search_types:
            start_time = time.time()
            df = pd.DataFrame()
            page = 1
            while True:
                url = f"https://www.gdt.gov.vn/TTHKApp/jsp/results.jsp?maTinh={tinh}&maHuyen={huyen}&maXa={xa}&kyLb=&searchType={type}&captcha={captcha}&pageNumber={page}"
                driver.get(url)

                notify_element = driver.find_element(By.XPATH, '/html/body/div[1]')
                if notify_element.text == "Không tìm thấy kết quả":
                    break

                table_element = driver.find_element(By.XPATH,search_types[type])
                tables = pd.read_html(table_element.get_attribute("outerHTML"))

                if len(tables[0]) < 5:
                    break
                tables[0]['mã_tỉnh'] = tinh
                tables[0]['mã_huyện'] = huyen
                tables[0]['mã_xã'] = xa
                tables[0]['search_type'] = type
                df=pd.concat([df,tables[0]])
                page += 1

            if type in ["11", "12", "03"]:
                df.columns = [x[0] + " " + x[1] if x[0] != x[1] else x[0] for x in df.columns]
            
            path1 = f'C:\\Users\\Admin\\PycharmProjects\\Crawl_data_gov\\BIDV_Data\\{tinh}_{huyen}_{xa}_{type}.csv'
            df.to_csv(path1,index=False,encoding='utf-8-sig')
            print(f"Done {tinh}_{huyen}_{xa}_{type}.csv")

            end_time = time.time()
            elapsed_time = end_time - start_time
            log_file = open("log.txt", "a")
            timestamp = time.strftime("%Y-%m-%d %H:%M:%S")

            number_of_records = len(df)
            number_of_distinct_period = df["Kỳ lập bộ"].nunique()
            number_of_unique_tax_code = df['Mã số thuế'].nunique()
            #number_of_distinct_period = 0
            #number_of_unique_tax_code = 0
            task_description_1 = f"Task Crawl {xa}"
            task_description_2 = f"Type: {type} - Number of records: {number_of_records} - Number of distinct period: {number_of_distinct_period} - Number of distinct tax code: {number_of_unique_tax_code} "
            log_message = f"{timestamp} - {task_description_1} - {elapsed_time:.2f} seconds - {task_description_2}\n"
            log_file.write(log_message)
            log_file.close()

            time.sleep(1)

Done 805_80519_8051905_11.csv
Done 805_80519_8051905_10.csv
Done 805_80519_8051905_12.csv
Done 805_80519_8051905_03.csv
Done 805_80519_8051905_04.csv


In [8]:
df_11 = pd.read_csv(r'C:\Users\Admin\PycharmProjects\Crawl_data_gov\BIDV_Data\805_80519_8051905_11.csv')
df_11[:5]

,STT,Họ tên,Mã số thuế,Kỳ lập bộ,Địa chỉ,Ngành nghề kinh doanh,Doanh thu khoán/tháng,Tiền thuế phải nộp trong tháng Tổng số Thuế,Tiền thuế phải nộp trong tháng Thuế GTGT,Tiền thuế phải nộp trong tháng Thuế TNCN,Tiền thuế phải nộp trong tháng Thuế TTĐB,Tiền thuế phải nộp trong tháng Thuế tài nguyên,Tiền thuế phải nộp trong tháng Thuế BVMT,Tiền thuế phải nộp trong tháng Phí BVMT,mã_tỉnh,mã_huyện,mã_xã,search_type
0,1,Bùi Văn Thành,1600887201,01/2016,"Tổ 9, ấp Thạnh Hòa, xã Bình Thạnh",Đóng tàu và thuyền;,37600000,1692000,1128000,564000,0,0,0,0,805,80519,8051905,11
1,2,Lâm Hoàng Mỹ Nhi,1601011618,01/2016,"Tổ 09, ấp Thạnh Hòa, xã Bình Thạnh","Bán lẻ thuốc, dụng cụ y tế, mỹ phẩm và vật phẩ...",29400000,441000,294000,147000,0,0,0,0,805,80519,8051905,11
2,3,Võ Thị Kiều Phương,1601017923,01/2016,"ấp Thạnh Hòa, xã Bình Thạnh",Bán lẻ hàng hóa khác lưu động hoặc tại chợ;,24600000,369000,246000,123000,0,0,0,0,805,80519,8051905,11
3,4,Hồ Chí Nghĩa,1600677677,01/2016,"ấp Thạnh Nhơn, xã Bình Thạnh",Bán lẻ hàng hóa khác trong các cửa hàng chuyên...,33400000,501000,334000,167000,0,0,0,0,805,80519,8051905,11
4,5,Cao Thắng Nhơn,8081407437,01/2016,"ấp Thạnh Hòa, xã Bình Thạnh-Xã Bình Thạnh-Huyệ...",NaN,21400000,321000,214000,107000,0,0,0,0,805,80519,8051905,11


In [38]:
df = pd.read_csv(r'C:\Users\Admin\PycharmProjects\Crawl_data_gov\Data\ds_full.csv')
print(len(df[df['maTinh'] == 701]))
ds_xa = df[df['maTinh'] == 701]['maXa'].values

358
